In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import time
import re
import plotly.express as px
import plotly.io as pio

pio.renderers.default = "notebook_connected"

In [ ]:
# URL base com filtros aplicados (início em 2000-07-26)
BASE_URL = "https://parlamento.gub.uy/noticiasyeventos/noticias?field_noticia_fecha_value%5Bmin%5D=2000-07-26&field_noticia_fecha_value%5Bmax%5D=&field_noticia_cuerpo_value=All&body_value="
DATABASE_NAME = "parlamento_uy.db"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

print("Configuração pronta! Filtros de data aplicados.")

In [ ]:
def create_database():
    conn = sqlite3.connect(DATABASE_NAME)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS articles (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT,
            date TEXT,
            category TEXT,
            url TEXT UNIQUE,
            source TEXT
        )
    """)
    conn.commit()
    conn.close()
    print("✅ Banco e tabela prontos")

create_database()

In [ ]:
def insert_article(title, date, category, url, source="Parlamento UY"):
    conn = sqlite3.connect(DATABASE_NAME)
    cursor = conn.cursor()
    try:
        cursor.execute("""
            INSERT INTO articles (title, date, category, url, source)
            VALUES (?, ?, ?, ?, ?)
        """, (title, date, category, url, source))
        conn.commit()
        return True
    except sqlite3.IntegrityError:
        return False
    finally:
        conn.close()

In [ ]:
def get_total_pages():
    try:
        print("Detectando número de páginas...")
        r = requests.get(BASE_URL, headers=HEADERS, timeout=30)
        soup = BeautifulSoup(r.text, "html.parser")
        
        # Tentativa 1: Busca pelo título 'Ir a la última página'
        # Exemplo HTML: <a href="..." title="Ir a la última página" ...>
        last_page_link = soup.find("a", title="Ir a la última página")
        
        # Tentativa 2: Busca por texto '»'
        if not last_page_link:
            last_page_link = soup.find("a", string=re.compile(r'»'))
            
        # Tentativa 3: Busca paginação genérica e pega o maior número
        if not last_page_link:
             print("Paginação 'última' não encontrada, buscando maior número nos links...")
             pagination_links = soup.select("ul.pagination li.page-item a.page-link")
             max_page = 0
             for link in pagination_links:
                 href = link.get('href', '')
                 match = re.search(r'page=(\d+)', href)
                 if match:
                     page_num = int(match.group(1))
                     if page_num > max_page:
                         max_page = page_num
             if max_page > 0:
                 return max_page

        if last_page_link:
            href = last_page_link['href']
            print(f"Link última encontrado: {href}")
            match = re.search(r'page=(\d+)', href)
            if match:
                return int(match.group(1))
        
        # Fallback manual em caso de falha na detecção automática
        # O usuário informou que espera cerca de 1113 páginas
        print("⚠️ Não foi possível determinar automaticamente. Usando fallback de 1113.")
        return 1113
        
    except Exception as e:
        print(f"Erro ao determinar total de páginas: {e}")
        return 1113 # Segura fallback

total_pages = get_total_pages()
print(f"Total de páginas detectadas para o período: {total_pages}")

In [ ]:
inserted_count = 0

# Itera de 0 até total_pages
for page in range(total_pages + 1):
    # Como BASE_URL já tem parametros, usamos &page=
    url = f"{BASE_URL}&page={page}"
    
    # Log simplificado a cada 10 páginas
    if page % 10 == 0 or page == total_pages:
        print(f"Coletando página {page}/{total_pages}...")
    
    try:
        r = requests.get(url, headers=HEADERS, timeout=30)
        if r.status_code != 200:
            print(f"  Status code {r.status_code} na página {page}, pulando...")
            continue
            
        soup = BeautifulSoup(r.text, "html.parser")
        
        articles = soup.find_all("article")
        
        # Se encontrar 0, avisar (pode ser problema de parsing ou fim real)
        if len(articles) == 0:
             print(f"⚠️ 0 notícias na página {page}. Verificando parsing...")
        
        for article in articles:
            # Título
            title_tag = article.select_one("h2.node__title a span")
            if not title_tag:
                title_tag = article.select_one("h2.node__title a")
            
            if not title_tag:
                continue
                
            title = title_tag.get_text(strip=True)
            
            # Link
            link_tag = article.select_one("h2.node__title a")
            link = f"https://parlamento.gub.uy{link_tag['href']}" if link_tag else ""
            
            # Data
            date_tag = article.select_one("time")
            date = date_tag.get_text(strip=True) if date_tag else ""
            
            # Categoria (Cuerpo)
            cat_tag = article.select_one(".field--name-field-noticia-cuerpo")
            category = cat_tag.get_text(strip=True) if cat_tag else "Parlamento"
            
            if insert_article(title, date, category, link):
                inserted_count += 1
        
        # Delay pequeno para não sobrecarregar
        time.sleep(0.5)
        
    except Exception as e:
        print(f"Erro na página {page}: {e}")
        time.sleep(5) # Espera maior em erro

print(f"\n📥 Total de novas notícias inseridas: {inserted_count}")

In [ ]:
def load_articles():
    conn = sqlite3.connect(DATABASE_NAME)
    df = pd.read_sql("SELECT * FROM articles", conn)
    conn.close()
    return df

df_db = load_articles()
print(f"📦 Total no banco: {len(df_db)} registros")
display(df_db.head())

In [ ]:
if not df_db.empty:
    # 1. Distribuição por Categoria
    cat_count = df_db['category'].value_counts().reset_index()
    cat_count.columns = ['Category', 'Count']
    
    fig1 = px.pie(cat_count, names='Category', values='Count', title='Notícias por Categoria (Cuerpo)')
    fig1.show()
    
    fig2 = px.bar(cat_count, x='Category', y='Count', title='Contagem por Categoria')
    fig2.show()
else:
    print("Sem dados para visualizar.")